In [1]:
# Libraries import
import sys
import os
import os.path as pth
sys.path.insert(0, '../..')
    
from lcav.lca_problem import *
from lcav.helpers import *
from lcav.io.configuration import *
from lcav.gui.plots import *
import matplotlib.pyplot as plt 
from premise import clear_cache

# Pretty print for Sympy
from sympy import init_printing
init_printing()

# Paths
#os.chdir('./notebook')
DATA_FOLDER = './data'
CONFIGURATION_FILE = pth.join(DATA_FOLDER, 'configuration_methodo_ei391.yaml')

#safe_delete_brightway_project('example_methodology_ei391')  # delete project for fresh restart
#clear_cache()  # clear premise cache

In [2]:
_, model, methods = LCAProblemConfigurator(CONFIGURATION_FILE).generate()

Initial setup of EcoInvent/Premise already done, skipping. To reset the project use option generate(reset=True).


In [3]:
process_tree(model, foreground_only=False, colormap='Pastel2')

In [4]:
agb.list_databases()

,backend,nb_activities,type
name,,,
biosphere3,sqlite,4718,biosphere
ecoinvent-3.9.1-cutoff,sqlite,21238,background
ecoinvent_cutoff_3.9_remind_SSP2-Base_2030,sqlite,29228,background
ecoinvent_cutoff_3.9_remind_SSP2-Base_2040,sqlite,29816,background
ecoinvent_cutoff_3.9_remind_SSP2-Base_2050,sqlite,30068,background
ecoinvent_cutoff_3.9_remind_SSP1-Base_2035,sqlite,29550,background
ecoinvent_cutoff_3.9_image_SSP2-Base_2040,sqlite,33643,background
Foreground DB,sqlite,21,foreground


In [5]:
agb.list_parameters()

group,name,label,default,min,max,std,distrib,unit,db
,aircraft_lifetime,aircraft lifetime,1.0,1,1,,linear,,Foreground DB
,airport_lifetime,airport lifetime,1.0,1,1,,linear,,Foreground DB
,amount_elec_h2,amount elec h2,1.0,1,1,,linear,,Foreground DB
,amount_h2,amount h2,1.0,1,1,,linear,,Foreground DB
,electricity_switch_param,electricity switch param,pv,,,,,,Foreground DB
,energy_density_h2,energy density h2,1.0,1,1,,linear,,Foreground DB
,energy_density_kero,energy density kero,1.0,1,1,,linear,,Foreground DB
,model,model,remind,,,,,,Foreground DB
,n_aircraft,n aircraft,1.0,1,1,,linear,,Foreground DB
,n_airports,n airports,1.0,1,1,,linear,,Foreground DB


In [6]:
list_processes(model, foreground_only=True)

,activity,unit,location,level,database,parent,exchange
model,model,None,GLO,0,Foreground DB,,
aircraft_production,aircraft_production,unit,GLO,1,Foreground DB,model,n_aircraft/aircraft_lifetime
"aircraft production, passenger aircraft, long haul (remind_SSP2-Base)","aircraft production, passenger aircraft, long ...",unit,GLO,2,Foreground DB,aircraft_production,model_remind*pathway_SSP2_Base
airports,airports,unit,GLO,1,Foreground DB,model,n_airports/airport_lifetime
market for airport (remind_SSP2-Base),market for airport (remind_SSP2-Base),unit,GLO,2,Foreground DB,airports,model_remind*pathway_SSP2_Base
energy,energy,kilowatt hour,GLO,1,Foreground DB,model,total_energy
kerosene,kerosene,None,GLO,2,Foreground DB,energy,(1 - share_efuel)/energy_density_kero
production,production,kilogram,GLO,3,Foreground DB,kerosene,1.00
market for kerosene (remind_SSP2-Base),market for kerosene (remind_SSP2-Base),kilogram,GLO,4,Foreground DB,production,model_remind*pathway_SSP2_Base
combustion,combustion,None,GLO,3,Foreground DB,kerosene,1.00


In [10]:
parameters = {
    'year': [2030, 2032, 2035, 2037, 2043],
    'model': 'remind',
    'pathway': 'SSP2_Base'
}

agb.compute_impacts(
            model,
            methods,
            **parameters
        )

,acidification - accumulated exceedance (AE)[mol H+-Eq],climate change - global warming potential (GWP100)[kg CO2-Eq],water use - user deprivation potential (deprivation-weighted water consumption)[m3 world eq. deprived],land use - soil quality index[dimensionless]
year,,,,
2030,4.79463e+07,1.22246e+10,4.1834e+09,2.06349e+11
2032,4.65241e+07,1.17268e+10,4.08841e+09,2.1031e+11
2035,4.43908e+07,1.09801e+10,3.94593e+09,2.16252e+11
2037,4.29686e+07,1.04823e+10,3.85095e+09,2.20214e+11
2043,3.95954e+07,9.24299e+09,3.61929e+09,2.29517e+11


In [3]:
test = {year: 'o' for year in [2030, 2040]}

In [4]:
test

{2030: 'o', 2040: 'o'}